# Data Analyzes and data in database (DuckDB)

# Data EDA

In [112]:
# Install libraries
# !pip3 install pandas_profiling
# !pip3 install duckdb==0.7.1

In [1]:
# Import necessary libraries
import pandas as pd
import glob
from ydata_profiling import ProfileReport
import numpy as np
import json
import duckdb
import string
import re
from textblob import TextBlob
import functions_project

# EDA imdb data

In [2]:
# Read imdb labeled train data
files = glob.glob(r'data/train*.csv')
imdb_review = pd.concat((pd.read_csv(file, on_bad_lines='skip', index_col=1) for file in files))
imdb_review = imdb_review.drop(columns=['Unnamed: 0'])

# Replace all missing values by legit missing values instead of string "\N"
imdb_review = imdb_review.replace('\\N', np.nan)

# Remove 13 rows (< 1% of all rows) where runtimeMinutes is nan
imdb_review = imdb_review[imdb_review['runtimeMinutes'].notna()]
imdb_review['runtimeMinutes'] = imdb_review['runtimeMinutes'].astype(int)

#replace startYear NA with the value of endYear
imdb_review['startYear']=imdb_review['startYear'].fillna(imdb_review['endYear'])

# Reset index so that every column is added to database
imdb_review = imdb_review.reset_index()
imdb_review

tconst                                   primaryTitle  \
0     tt0010600                                       The Doll   
1     tt0011841                                  Way Down East   
2     tt0012494                                        Déstiny   
3     tt0015163                                  The Navigator   
4     tt0016220                       The Phantom of the Opera   
...         ...                                            ...   
7941  tt9625664                                  Trauma Center   
7942  tt9741310                                          Slaxx   
7943  tt9742392                                        Kindred   
7944  tt9850386  The Bee Gees: How Can You Mend a Broken Heart   
7945  tt9911196                            The Marriage Escape   

                       originalTitle startYear endYear  runtimeMinutes  \
0                          Die Puppe      1919     NaN              66   
1                      Way Down East      1920     NaN             145   
2                       Der müde Tod      1921     NaN              97   
3                      The Navigator      1924     NaN              59   
4           The Phantom of the Opera      1925     NaN              93   
...                              ...       ...     ...             ...   
7941                             NaN      2019     NaN              87   
7942                           Slaxx      2020     NaN              77   
7943                         Kindred      2020     NaN             101   
7944                             NaN      2020     NaN             111   
7945  De beentjes van Sint-Hildegard      2020     NaN             103   

      numVotes  label  
0       1898.0   True  
1       5376.0   True  
2       5842.0   True  
3       9652.0   True  
4      17887.0   True  
...        ...    ...  
7941   12951.0  False  
7942    2464.0  False  
7943    1719.0  False  
7944    4144.0   True  
7945    3242.0   True  

[7946 rows x 8 columns]

In [3]:
# Call function map_cleaned_strings to apply in the dataframe on the column primary title
functions_project.map_cleaned_strings(imdb_review, 'primaryTitle')

imdb_review

tconst                                  primaryTitle  \
0     tt0010600                                      the doll   
1     tt0011841                                 way down east   
2     tt0012494                                       destiny   
3     tt0015163                                 the navigator   
4     tt0016220                      the phantom of the opera   
...         ...                                           ...   
7941  tt9625664                                 trauma center   
7942  tt9741310                                         slaxx   
7943  tt9742392                                       kindred   
7944  tt9850386  the bee gees how can you mend a broken heart   
7945  tt9911196                           the marriage escape   

                       originalTitle startYear endYear  runtimeMinutes  \
0                          Die Puppe      1919     NaN              66   
1                      Way Down East      1920     NaN             145   
2                       Der müde Tod      1921     NaN              97   
3                      The Navigator      1924     NaN              59   
4           The Phantom of the Opera      1925     NaN              93   
...                              ...       ...     ...             ...   
7941                             NaN      2019     NaN              87   
7942                           Slaxx      2020     NaN              77   
7943                         Kindred      2020     NaN             101   
7944                             NaN      2020     NaN             111   
7945  De beentjes van Sint-Hildegard      2020     NaN             103   

      numVotes  label  
0       1898.0   True  
1       5376.0   True  
2       5842.0   True  
3       9652.0   True  
4      17887.0   True  
...        ...    ...  
7941   12951.0  False  
7942    2464.0  False  
7943    1719.0  False  
7944    4144.0   True  
7945    3242.0   True  

[7946 rows x 8 columns]

In [4]:
#check if - is also out
for i in imdb_review['primaryTitle']:
    if '-' in i:
        print(i)

In [5]:
# textblob on primarytitle. 
imdb_review['primaryTitle_2'] = imdb_review['primaryTitle'].apply(lambda x:str(TextBlob(x)))
imdb_review

tconst                                  primaryTitle  \
0     tt0010600                                      the doll   
1     tt0011841                                 way down east   
2     tt0012494                                       destiny   
3     tt0015163                                 the navigator   
4     tt0016220                      the phantom of the opera   
...         ...                                           ...   
7941  tt9625664                                 trauma center   
7942  tt9741310                                         slaxx   
7943  tt9742392                                       kindred   
7944  tt9850386  the bee gees how can you mend a broken heart   
7945  tt9911196                           the marriage escape   

                       originalTitle startYear endYear  runtimeMinutes  \
0                          Die Puppe      1919     NaN              66   
1                      Way Down East      1920     NaN             145   
2                       Der müde Tod      1921     NaN              97   
3                      The Navigator      1924     NaN              59   
4           The Phantom of the Opera      1925     NaN              93   
...                              ...       ...     ...             ...   
7941                             NaN      2019     NaN              87   
7942                           Slaxx      2020     NaN              77   
7943                         Kindred      2020     NaN             101   
7944                             NaN      2020     NaN             111   
7945  De beentjes van Sint-Hildegard      2020     NaN             103   

      numVotes  label                                primaryTitle_2  
0       1898.0   True                                      the doll  
1       5376.0   True                                 way down east  
2       5842.0   True                                       destiny  
3       9652.0   True                                 the navigator  
4      17887.0   True                      the phantom of the opera  
...        ...    ...                                           ...  
7941   12951.0  False                                 trauma center  
7942    2464.0  False                                         slaxx  
7943    1719.0  False                                       kindred  
7944    4144.0   True  the bee gees how can you mend a broken heart  
7945    3242.0   True                           the marriage escape  

[7946 rows x 9 columns]

In [6]:
# Perform EDA report on imdb reviews
profile = ProfileReport(imdb_review, title="Report")
profile

# EDA remarks
# remove original title? >50% missing values
# remove endYear? >90% missing values
# some titles are duplicate, problem? I don't think so
# what about outliers (number of votes)

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [118]:
# highly rated movies mostly have more runtime minutes and have more number of votes
imdb_review.groupby('label').mean()

/var/folders/p3/prw8fq7d4t350r6vj8kcn_rh0000gn/T/ipykernel_26423/1801606364.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  imdb_review.groupby('label').mean()


runtimeMinutes      numVotes
label                              
False       97.986872  10053.794026
True       113.341029  48761.503880

## read json files

In [7]:
# Load mapping table movie to writer
movie_writer = pd.read_json('data/writing.json')

# Replace missing values by legit missing values
movie_writer = movie_writer.replace('\\N', np.nan)

# Create an EDA report
profile = ProfileReport(movie_writer, title="Report")
profile

# EDA remarks
# Some movies have multiple writers. Movie tt0401711 even has 29 writers. How are we going to map this? 
# What to do with rows with missing writer value? I think we should remove those rows.

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
# Load mapping table movie to director
directing = pd.read_json('data/directing.json')

# Replace missing values by legit missing values
directing = directing.replace('\\N', np.nan)

# Create an EDA report
profile = ProfileReport(directing, title="Report")
profile

# # EDA remarks
# # Some movies have multiple directors. Movie tt1687247 even has 35 writers. How are we going to map this? 
# # What to do with rows with missing director value? I think we should remove those rows.

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
# Remove movies with missing director (= 2 movies)
directing = directing.dropna()

# Remove movies with missing writer (= 297 movies)
movie_writer = movie_writer.dropna()


# # Transform data and remove outliers before pushing data into the database
# # Select first 5 directors (only <1% of the movies have more than 5 directors, see notebook 1.A)
# # Transform data so that the directors are grouped per movie.
# # Reset index so that every column is added to database
# top_5_directors_per_movie = directing.groupby('movie').head(5).groupby(['movie'], as_index=False).agg({'director': ', '.join})
# movies_and_directors = pd.concat([top_5_directors_per_movie['movie'], top_5_directors_per_movie['director'].str.split(', ', expand=True)], axis=1)
# movies_and_directors = movies_and_directors.rename(columns={0: "director_1", 1: "director_2", 2: "director_3", 3: "director_4", 4: "director_5"})




# # Transform data and remove outliers before pushing data into the database
# # Select first 8 writers (only <1% of the movies have more than 8 writers, see notebook 1.B)
# # Transform data so that the writers are grouped per movie.
# # Reset index so that every column is added to database
# top_8_writers_per_movie = movie_writer.groupby('movie').head(8).groupby(['movie'], as_index=False).agg({'writer': ', '.join})
# movie_and_writers = pd.concat([top_8_writers_per_movie['movie'], top_8_writers_per_movie['writer'].str.split(', ', expand=True)], axis=1)
# movie_and_writers = movie_and_writers.rename(columns={0: "writer_1", 1: "writer_2", 2: "writer_3", 3: "writer_4", 4: "writer_5", 5: "writer_6", 6: "writer_7", 7: "writer_8"})


In [10]:
#dummie tables
# get top 10 most frequent names for director and writer 

##director
n = 5
inlist = directing['director'].value_counts()[:n].index.tolist()

directing_top_5_most = directing[directing[['director']].isin(inlist).any(1)]
#dummie table
dummie_director = pd.get_dummies(directing_top_5_most, columns=['director'])
#rename columns
dummie_director = dummie_director.set_axis(['movie','director_1_dum', 'director_2_dum', 'director_3_dum', 'director_4_dum', 'director_5_dum'], axis=1, inplace=False)

##writer
n = 5
inlist = movie_writer['writer'].value_counts()[:n].index.tolist()

writer_top_5_most = movie_writer[movie_writer[['writer']].isin(inlist).any(1)]
#dummie table
dummie_writer = pd.get_dummies(writer_top_5_most, columns=['writer'])
#rename columns
dummie_writer = dummie_writer.set_axis(['movie','writer_1_dum', 'writer_2_dum', 'writer_3_dum', 'writer_4_dum', 'writer_5_dum'], axis=1, inplace=False)
dummie_writer

movie  writer_1_dum  writer_2_dum  writer_3_dum  writer_4_dum  \
184    tt0018526             0             0             0             1   
384    tt0023940             0             0             0             1   
438    tt0024481             0             0             0             1   
699    tt0029442             0             0             0             1   
802    tt0031381             0             0             0             1   
...          ...           ...           ...           ...           ...   
19641  tt4476736             0             1             0             0   
20500  tt5662932             0             0             0             0   
20822  tt6129302             0             0             0             0   
20992  tt6409416             0             0             0             0   
21356  tt7180544             0             0             0             0   

       writer_5_dum  
184               0  
384               0  
438               0  
699               0  
802               0  
...             ...  
19641             0  
20500             1  
20822             1  
20992             1  
21356             1  

[99 rows x 6 columns]

In [ ]:
dummie_all_director = pd.get_dummies(directing, columns=['director'])
dummie_all_director = dummie_director.groupby('movie').sum()
dummie_all_director = dummie_all_director.reset_index()
dummie_all_director

In [ ]:
dummie_all_writers = pd.get_dummies(movie_writer, columns=['writer'])
dummie_all_writers = dummie_all_writers.groupby('movie').sum()
dummie_all_writers = dummie_all_writers.reset_index()
dummie_all_writers

In [125]:
# create a connection to a file called 'imdb_reviews.db'
con = duckdb.connect('imdb_reviews.db')

# create a table and load data into it

con.sql("DROP TABLE IF EXISTS imdb_review_train")
con.sql("DROP TABLE IF EXISTS dummie_director_all")
con.sql("DROP TABLE IF EXISTS dummie_writer_all")
con.sql("DROP TABLE IF EXISTS dummie_director_top_5")
con.sql("DROP TABLE IF EXISTS dummie_writer_top_5")

con.sql("CREATE TABLE IF NOT EXISTS imdb_review_train AS SELECT * FROM imdb_review")
con.sql("CREATE TABLE IF NOT EXISTS dummie_director_all AS SELECT * FROM dummie_all_director")
con.sql("CREATE TABLE IF NOT EXISTS dummie_writer_all AS SELECT * FROM dummie_all_writers")
con.sql("CREATE TABLE IF NOT EXISTS dummie_director_top_5 AS SELECT * FROM dummie_director")
con.sql("CREATE TABLE IF NOT EXISTS dummie_writer_top_5 AS SELECT * FROM dummie_writer")

con.close()